# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [156]:
import numpy as np 
from collections import deque 
import itertools
import functools
import resource
import sys

In [157]:
resource.setrlimit(resource.RLIMIT_STACK, (2**40,-1))
sys.setrecursionlimit(10**9)

In [158]:
class Node(): 
    def __init__ (self, board, parent): 
        self.board = board 
        self.signature = self.get_signature() 
        self.parent = parent
        self.empty_pos = self.get_empty_pos() 
        self.children = []

    def get_signature (self): 
        flattenBoard = list(itertools.chain.from_iterable(self.board)) 
        listToSignature = lambda a, b: str(a) + str(b)

        return functools.reduce(listToSignature, flattenBoard)

    def get_possible_moves (self): 
        all_moves = [
            (0, 1),
            (1, 0),
            (0, -1), 
            (-1, 0)
            ] 
        
        x, y = self.empty_pos 
        possibleMoves = list() 

        for xDelta, yDelta in all_moves: 
            newX = x + xDelta
            newY = y + yDelta 
            
            if (-1 < newX < 3) and ((-1 < newY < 3)):
                move = (newX, newY) 
                possibleMoves.append(move) 
        
        return possibleMoves 

 
    def apply_move (self, move): 
        new_board = self.board.copy()

        new_board[self.empty_pos], new_board[move] = new_board[move], new_board[self.empty_pos]
        return new_board

    def get_children (self): 
        children = [] 
        possible_moves = self.get_possible_moves() 

        for move in possible_moves: 
            child_board = self.apply_move(move)  
            child = Node(child_board, self)

            children.append(child) 

        self.children = children 

    def get_empty_pos (self): 
        y, x = np.where(self.board == 0)
        x = x.item() 
        y = y.item()
     
        return (y, x)

In [159]:
def print_solution (initial_node, end_node): 
  solution = []
  while not (end_node.parent == None): 
    solution.append(end_node.board)
    end_node = end_node.parent 
  
  solution = solution[::-1]

  print(initial_node.board)
  print("")

  for board in solution:
    print(board)
    print("")


## Busca em largura

In [160]:
def bfs(initial_node):
    solution_signature = "123456780"
    visited_nodes = set() 

    if initial_node.signature == solution_signature:
        return initial_node
  
    q = deque()
    q.append(initial_node)
    visited_nodes.add(initial_node)

    while (len(q) > 0):
        q[0].get_children()

        for i in q[0].children:
            if i in visited_nodes: 
              continue
            visited_nodes.add(i)
            if i.signature == solution_signature:
                return i
            q.append(i)
        q.popleft()

In [161]:
board =np.array([[4, 5, 1],
                 [7, 0, 2],
                 [8, 6, 3]])

In [162]:
initial_node = Node(board=board, parent=None)
end_node = bfs(initial_node)

In [163]:
print_solution(initial_node, end_node)

[[4 5 1]
 [7 0 2]
 [8 6 3]]

[[4 0 1]
 [7 5 2]
 [8 6 3]]

[[4 1 0]
 [7 5 2]
 [8 6 3]]

[[4 1 2]
 [7 5 0]
 [8 6 3]]

[[4 1 2]
 [7 5 3]
 [8 6 0]]

[[4 1 2]
 [7 5 3]
 [8 0 6]]

[[4 1 2]
 [7 5 3]
 [0 8 6]]

[[4 1 2]
 [0 5 3]
 [7 8 6]]

[[0 1 2]
 [4 5 3]
 [7 8 6]]

[[1 0 2]
 [4 5 3]
 [7 8 6]]

[[1 2 0]
 [4 5 3]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]



## Busca em Profundidade

In [164]:
solution_signature = "123456780"
visited_nodes = set() 

def dfs (node): 
  if node.signature == solution_signature: 
    return node
    
  visited_nodes.add(node.signature) 

  node.get_children() 
  for child in node.children: 
    if child.signature in visited_nodes: 
      continue 
    solution = dfs(child)
    if solution is not None: 
     return solution

In [165]:
board =np.array([[1, 5, 2],
                 [4, 0, 3],
                 [7, 8, 6]])

In [166]:
initial_node = Node(board=board, parent=None)
end_node = dfs(initial_node)

In [167]:
print_solution(initial_node, end_node)

[[1 5 2]
 [4 0 3]
 [7 8 6]]

[[1 5 2]
 [4 3 0]
 [7 8 6]]

[[1 5 2]
 [4 3 6]
 [7 8 0]]

[[1 5 2]
 [4 3 6]
 [7 0 8]]

[[1 5 2]
 [4 3 6]
 [0 7 8]]

[[1 5 2]
 [0 3 6]
 [4 7 8]]

[[1 5 2]
 [3 0 6]
 [4 7 8]]

[[1 5 2]
 [3 6 0]
 [4 7 8]]

[[1 5 2]
 [3 6 8]
 [4 7 0]]

[[1 5 2]
 [3 6 8]
 [4 0 7]]

[[1 5 2]
 [3 6 8]
 [0 4 7]]

[[1 5 2]
 [0 6 8]
 [3 4 7]]

[[1 5 2]
 [6 0 8]
 [3 4 7]]

[[1 5 2]
 [6 8 0]
 [3 4 7]]

[[1 5 2]
 [6 8 7]
 [3 4 0]]

[[1 5 2]
 [6 8 7]
 [3 0 4]]

[[1 5 2]
 [6 8 7]
 [0 3 4]]

[[1 5 2]
 [0 8 7]
 [6 3 4]]

[[1 5 2]
 [8 0 7]
 [6 3 4]]

[[1 5 2]
 [8 7 0]
 [6 3 4]]

[[1 5 2]
 [8 7 4]
 [6 3 0]]

[[1 5 2]
 [8 7 4]
 [6 0 3]]

[[1 5 2]
 [8 7 4]
 [0 6 3]]

[[1 5 2]
 [0 7 4]
 [8 6 3]]

[[1 5 2]
 [7 0 4]
 [8 6 3]]

[[1 5 2]
 [7 4 0]
 [8 6 3]]

[[1 5 2]
 [7 4 3]
 [8 6 0]]

[[1 5 2]
 [7 4 3]
 [8 0 6]]

[[1 5 2]
 [7 4 3]
 [0 8 6]]

[[1 5 2]
 [0 4 3]
 [7 8 6]]

[[0 5 2]
 [1 4 3]
 [7 8 6]]

[[5 0 2]
 [1 4 3]
 [7 8 6]]

[[5 2 0]
 [1 4 3]
 [7 8 6]]

[[5 2 3]
 [1 4 0]
 [7 8 6]]

[[5 2 3]
 [1 4

## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



In [ ]:
BFS:
      Por se tratar de um algorítmo que visita e armazena possivelmente todos os estados de uma árvore ( O(V+E) ), seu custo de memória é grande.
    Em casos complexos, ele estorou os 12GB concedidos pelo colab. Sua eficiência também depende muito da maneira como os dados estão,
    estruturados, futuramente explorarei melhores maneiras de representar um estado. Seu processamento não envolve operações matemáticas
    complexas, embora haja uma pequena operação matricial, que poderia ser evitada e a criação dos filhos de um estado poderia ser parale
    lizada 

In [ ]:
DFS: 
      Como já esperado, nem sempre o algoritmo encontrou o caminho mais razo, como o BFS faz, porém os encontra a um custo de memória muito mais 
    acessível, dado que armazena apenas um grafo, que contém seu pai, sendo assim, foi capaz de resolver qualquer um dos tabuleiros sem estourar o 
    limite de memória
